In [1]:
import tensorflow as tf
import keras 
from tqdm import tqdm
import re
import pandas as pd 
import numpy as np
import random as rn
print(keras.__version__)
print(tf.__version__)

# Use a fixed seed for the random number generator to address randomness problem and get reproducable results with keras. the numbers don't make much difference.
seed = 42
np.random.seed(seed)

2.3.1
1.15.0


Using TensorFlow backend.


## Preprocessing

In [2]:
from tqdm import tqdm


sentences=[]
words = []
targets=[] 
sen_position=0 

labels= {'Correct':0, 'ArtOrDet':1, 'Nn':2, 'Prep':3, 'SVA':4, 'Vform':5, 'Vt':6, 'Wform':7}

 
# Import the sentences from the data file
f=open('/Users/highsierra/Tech-Skills/Unorganised/conll2014-master/release3.2/data/conll14st-preprocessed.m2')
for line in tqdm(f):
    parts = line.split()
    if(len(parts)>0):
        if line[0]=='S':
            # Cleaning
            line = line[2:].strip()
            
            # List of sentences
            sentences.append(line)

            # List of words, for the vocabulary info
            words = words + line.split()

            
            # Initial Outputs Processing  
            
            # By default, consider every word as non-erroneous, by creating an array with the tag "Correct" per every word.
            tags=np.zeros(shape=(len(parts)-1), dtype='int32')
            # Combine the tags associated with each sentence vertically in order to allign them with the input words
            targets.append(tags)
            # Keep track of the sentence's position
            sen_position += 1
            
        elif parts[0]=='A':
            if re.findall("ArtOrDet", parts[2]) or re.findall("Nn", parts[2]) or re.findall("Vt", parts[2]) or re.findall("Prep", parts[2]) or re.findall("Vform", parts[2]) or re.findall("Wform", parts[2]) or re.findall("SVA", parts[2]):
                # Keep track of the erroneous word's position by extracting it from the sentence annotation 
                digit = [int(j) for j in re.findall("[0-9]+", parts[2][:2])]            
    
                for key in labels:
                    if  re.search(key, parts[2]):
                        err = labels.get(key)
                 
                # Using its extracted position, place the erroneous word's tag in its sentence
                targets[sen_position - 1][digit[0]-1] = err

158784it [12:46, 207.17it/s]


In [3]:
sen_len = []
for sen in sentences:
    sen_len.append(len(sen.split()))
    
MAX_SEQUENCE_LENGTH = max(sen_len)
print(MAX_SEQUENCE_LENGTH) #print(leng.index(max(sen_len)))

222


In [4]:
MAX_VOCAB_SIZE = len(set(words))
print(MAX_VOCAB_SIZE)

33762


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, filters='\t\n')
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
word2idx = tokenizer.word_index

In [7]:
X = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', value=0)

In [8]:
y = pad_sequences(targets, maxlen=MAX_SEQUENCE_LENGTH, padding='post', value=0)

In [9]:
print(X.shape)
print(y.shape)

(57151, 222)
(57151, 222)


## Modeling

In [10]:
# Partition sentences it into training and test sets: 80%, 20%

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=seed)

In [11]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout,Reshape, SimpleRNN, Bidirectional
from keras.utils import to_categorical

In [22]:
def Model_(x_tr, y_tr, epos=3, my_batch_size=32):  
    input = Input(shape=(MAX_SEQUENCE_LENGTH,)) # This returns a tensor. The comma is necessary when you have only one dimension.
    model = Embedding(input_dim=MAX_VOCAB_SIZE, output_dim=50, input_length=MAX_SEQUENCE_LENGTH, name="Embedding")(input) # output_dim if 50 then the nw will learn 50-dimentional embeddings for each word. # This embedding layer will encode the input sequence into a sequence of dense 50-dimensional vectors.
    model = SimpleRNN(units=50, return_sequences=True, recurrent_dropout=0.2)(model)
    model = Dropout(0)(model)
    out = TimeDistributed(Dense(8, activation='softmax'))(model)
    model = Model(input, out)
    
    return model

In [23]:
model = Model_(X_train,  y_train)

In [24]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
#For training and testing the network we also need to change the labels y to categorial.

ycat_train = to_categorical(y_train, num_classes=8)
ycat_test = to_categorical(y_test, num_classes=8)

In [28]:
model.fit(X_train, ycat_train, epochs=3, batch_size=32, verbose=1) 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/3
45720/45720 [==============================] - 388s 8ms/step - loss: 0.0406 - accuracy: 0.9939
Epoch 2/3
45720/45720 [==============================] - 396s 9ms/step - loss: 0.0127 - accuracy: 0.9983
Epoch 3/3
45720/45720 [==============================] - 554s 12ms/step - loss: 0.0102 - accuracy: 0.9983


## Evaluation
Calculate the following metrics:
1. Accuracy
2. Precision
3. Recall
4. F1 

In [29]:
print(model.metrics_names)

['loss', 'accuracy']


In [30]:
scores = model.evaluate(X_test, ycat_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

11431/11431 [==============================] - 26s 2ms/step
Accuracy: 99.83%


In [31]:
pred = model.predict(X_test, verbose=1) 
print(pred.shape)

11431/11431 [==============================] - 21s 2ms/step
(11431, 222, 8)


In [32]:
y_pred = np.argmax(pred,axis=-1)

In [33]:
from sklearn.metrics import precision_score, recall_score, f1_score


pres_score = []

for tru,pred in zip (y_test, y_pred):
    pres_score.append(precision_score(tru,pred,average='macro'))

precision = np.mean(pres_score)
print(precision)

/opt/miniconda3/envs/tensorflow1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.863285815158046


In [34]:
rec_score = []

for tru,pred in zip (y_test, y_pred):
    rec_score.append(recall_score(tru,pred,average='macro'))

recall= np.mean(rec_score)
print(recall)

/opt/miniconda3/envs/tensorflow1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.8639421058768345


In [35]:
# Manually, it's calculated according to the formula:  f1 = (2 * precision * recall) / (precision + recall)
f_score = []

for tru,pred in zip (y_test, y_pred):
    f_score.append(f1_score(tru,pred,average='macro'))
f1 = np.mean(f_score)
print(f1)

0.8636089662290262


### Example of the experiments or runs using a SimpleRNN architecture:

Experiments for: Finding the optimal Dropout value from the list [0, 0.1, 0.2]
#### Run 1:  (a.k.a this one)
###### The Variable: 
    Dropout = 0 
    
###### Defaults:
    Number of layers = 1
    Number of units or neurons within a layer = 50
    Embedding size = 50
    Batch size = 32
    
###### Constants:
       Recurrent Dropout = 0.2
       
#### Run 2:  (a.k.a the next execution)
###### The Variable: 
    Dropout = 0.1 
    
###### Defaults:
    same as previous values
    
###### Constants:
    same as previous values
     
#### Run 3:  (a.k.a the final execution, or the experiment using the last proposed value for Dropout)
###### The Variable: 
    Dropout = 0.2 
    
###### Defaults:
    same as previous values
    
###### Constants:
    same as previous values